#### Query
```
SELECT *
FROM mean_param
WHERE sigma IN (3, 4) -- Selecting PCEBs (sigma=3 or sigma=4)
AND type = 'WD/M';

```

In [1]:
import csv
import lightkurve as lk
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import curve_fit
import numpy as np

In [2]:
palette = sns.color_palette()

In [3]:
# Define the sine wave function
def sine_wave(t, amplitude, frequency, phase, offset):
    t_float = t.value.astype(float)
    return amplitude * np.sin(2 * np.pi * frequency * t_float + phase) + offset

In [4]:
# Define a function to fold the light curve
def fold_light_curve(time, flux, period):
    folded_time = time % period
    # Sort the folded time for plotting
    sorted_indices = np.argsort(folded_time)
    return folded_time[sorted_indices], flux[sorted_indices]

In [5]:
# Make query seperated by commas instead of spaces
with open('sdss_query.csv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter='\t')

    with open('final_sdds_query.csv', 'w', newline='') as outfile:
        writer = csv.writer(outfile)
        
        # Iterate through each row
        for row in reader:
            # Replace spaces for commas for each element
            modified_row = [column.replace(' ', ',') for column in row]
            
            writer.writerow(modified_row)

## SDSSJ013851.54-001621.6
*row 1*

In [ ]:
star1_result = lk.search_lightcurve('SDSSJ013851.54-001621.6', mission='TESS')
star1_result

In [ ]:
star1_lc = star1_result[0].download().remove_nans()
star1_lc.plot(color=palette[0])
plt.show()

In [ ]:
star1_pg = star1_lc.to_periodogram(oversample_factor=1)
star1_pg.plot(color = palette[0]);
plt.show()

In [ ]:
star1_pg.period

In [ ]:
period_fold_lc = star1_lc.fold(period=star1_pg.period_at_max_power).scatter(color = palette[0]);
plt.show()

In [ ]:
star1_pg.period_at_max_power

In [ ]:
star1_lc.fold(period=star1_pg.period_at_max_power, wrap_phase=0.01).scatter(color = palette[0]);
plt.show()

In [ ]:
folded_lc = star1_lc.fold(period=star1_pg.period_at_max_power, wrap_phase=0.01)

initial_guess = (24, 1, 0, 0) # amplitude, frequency, phase, offset
popt, pcov = curve_fit(sine_wave, folded_lc.time, folded_lc.flux, p0=initial_guess)

fitted_curve = sine_wave(folded_lc.time, *popt)
time_values = folded_lc.time.value.astype(float)

plt.figure(figsize=(14,7))
plt.scatter(time_values, folded_lc.flux, color=palette[0], label='Folded Light Curve')
plt.plot(time_values, fitted_curve, color=palette[1], label='Fitted Sine Wave')
plt.xlabel('Phase')
plt.ylabel('Flux')
plt.legend()
plt.show()

In [ ]:
"""
    FOLD SIN WITH OG DATA!
"""

## SDSSJ084841.17+232051.7
*row 5*

In [ ]:
star2_result = lk.search_lightcurve('SDSSJ084841.17+232051.7', mission='TESS')
star2_result

In [ ]:
star2_lc = star2_result.download()
star2_lc.plot()
plt.show()

In [ ]:
star2_lc.scatter()
plt.show()